In [1]:
from utils import read_config, OracleAgent

In [2]:
configs = read_config(".env/info.json")
# BIDB = configs['BIDB_conn_info']
# bi_agent = OracleAgent(BIDB)
DWDB = configs['DW_conn_info']
dw_agent = OracleAgent(DWDB)

In [3]:
query = """
    SELECT * FROM ALL_Views
    where owner = 'YFYDW'
"""

dw_agent.read_table(query=query)

,owner,view_name,text_length,text,text_vc,type_text_length,type_text,oid_text_length,oid_text,view_type_owner,view_type,superview_name,editioning_view,read_only,container_data,bequeath,origin_con_id
0,YFYDW,WACES_CARBON_EMS01_DF_V,285,"SELECT\n ORG.ORG_NAME,\n CHECKTYPE_CODE,...","SELECT\n ORG.ORG_NAME,\n CHECKTYPE_CODE,...",None,None,None,None,None,None,None,N,N,N,DEFINER,0
1,YFYDW,WACES_CARBON_EMS02_MF_V,611,"select \n org.org_name,\n c.check_date p...","select \n org.org_name,\n c.check_date p...",None,None,None,None,None,None,None,N,N,N,DEFINER,0
